In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch



2025-09-02 13:28:37.962720: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756819718.116639      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756819718.159195      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import random
import os
import numpy as np
import torch

def set_seed(seed: int = 42):
    """
    Set seed for reproducibility across Python, NumPy, and PyTorch.
    Ensures deterministic behavior where possible.
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU

    # Deterministic settings for PyTorch
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed
SEED = 42
set_seed(SEED)

# Optional: confirm
print("Seed set to:", SEED)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.current_device(), torch.cuda.get_device_name(0))


Seed set to: 42
CUDA available: True
Device: 0 Tesla T4


In [ ]:

train = pd.read_csv('/kaggle/input/map-charting-student-math-misunderstandings/train_Clean.csv')
train = pd.DataFrame(train)

In [ ]:
train.head(2)

In [ ]:
train['Misconception'].unique()

In [ ]:
# containace of the misconception

import pandas as pd
import matplotlib.pyplot as plt

object_cols = ['Misconception']

for col in object_cols:
    plt.figure(figsize=(8,4))
    train[col].value_counts().plot(kind="bar")
    plt.title(f"Frequency of values in '{col}'")
    plt.xlabel("Category")
    plt.ylabel("Count")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()


In [ ]:
# we will firstly fine tune a bnary classification model on questionText and  MC_answer 
# to predict weathr it is tre or false
# then we will train another model on questiontext, MC-answer, and studentExplanation to predict
# 3 classes misconception, correct, neither (or 0,1,2 then we will map this)
# Again we train a model for multi classification of the Misconceptions, the model will be fine tuned on
# questionText, MC_answer, (weather correct, neither or miconception), (weather true or false), studentExplanation

# Finall we will join all the predictions according to the submition format e.g True_Neither- NA


In [ ]:
train.head(2)

In [ ]:
test = pd.read_csv('/kaggle/input/map-charting-student-math-misunderstandings/test.csv')
test.head(2)

In [ ]:
submission = pd.read_csv('/kaggle/input/map-charting-student-math-misunderstandings/submission.csv')
submission.head(2)

In [ ]:
pip uninstall transformers


In [ ]:
pip cache purge


In [ ]:
pip install transformers

In [ ]:
!pip install --upgrade transformers


In [ ]:
%pip install --upgrade transformers datasets

In [ ]:
'''
train = pd.read_csv("/kaggle/input/map-charting-student-math-misunderstandings/train_Clean.csv")
test = pd.read_csv("/kaggle/input/map-charting-student-math-misunderstandings/test.csv")
sample_sub = pd.read_csv("/kaggle/input/map-charting-student-math-misunderstandings/submission.csv")

print(train.head())
print(test.head())


# Step 1: Binary labels (True/False)
train["binary_label"] = train["Category"].apply(lambda x: 1 if "True" in str(x) else 0)

# Step 2: Ternary labels (Correct, Neither, Misconception)
# Step 2: Ternary labels (Correct, Neither, Misconception)
def ternary_mapper(x):
    x = str(x)  # make sure it's a string
    if "Correct" in x: 
        return 1
    elif "Neither" in x: 
        return 2
    elif "Misconception" in x: 
        return 0
    else: 
        return -1

train["ternary_label"] = train["Category"].apply(ternary_mapper)

# Step 3: Misconception labels
misconception_encoder = LabelEncoder()
train["misconception_label"] = misconception_encoder.fit_transform(
    train["Misconception"].fillna("NA")
)

num_misconceptions = len(misconception_encoder.classes_)
print("Number of misconception classes:", num_misconceptions)


from datasets import Dataset

def make_dataset(df, inputs, label_col):
    texts = df[inputs].astype(str).agg(" ".join, axis=1).tolist()
    labels = df[label_col].tolist()
    return Dataset.from_dict({"text": texts, "label": labels})

# Step 1 dataset
ds_binary = make_dataset(train, ["QuestionText", "MC_Answer"], "binary_label")

# Step 2 dataset
ds_ternary = make_dataset(train, ["QuestionText", "MC_Answer", "StudentExplanation"], "ternary_label")

# Step 3 dataset
ds_misconception = make_dataset(train, ["QuestionText", "MC_Answer", "StudentExplanation"], "misconception_label")



MODEL_NAME = "distilbert-base-uncased"  # small + fast
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

ds_binary = ds_binary.map(tokenize_function, batched=True)
ds_ternary = ds_ternary.map(tokenize_function, batched=True)
ds_misconception = ds_misconception.map(tokenize_function, batched=True)

ds_binary.set_format("torch")
ds_ternary.set_format("torch")
ds_misconception.set_format("torch")


def train_model(dataset, num_labels, output_dir):
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)

    training_args = TrainingArguments(
        output_dir=output_dir,
        do_eval=True,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=2,
        weight_decay=0.01,
        logging_steps=100,
        disable_tqdm=False   # ✅ ensure progress bar is shown

    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        eval_dataset=dataset,  # TODO: add real val split
        tokenizer=tokenizer
    )

    trainer.train()
    return trainer


binary_trainer = train_model(ds_binary, num_labels=2, output_dir="./binary_model")
ternary_trainer = train_model(ds_ternary, num_labels=3, output_dir="./ternary_model")
misconception_trainer = train_model(ds_misconception, num_labels=num_misconceptions, output_dir="./mis_model")

'''

In [3]:


# Load CSV
train = pd.read_csv("/kaggle/input/map-charting-student-math-misunderstandings/train_Clean.csv")

# Labels
train["binary_label"] = train["Category"].apply(lambda x: 1 if str(x).split("_")[0].lower() == "true" else 0)

# Dataset
texts = (train["QuestionText"].astype(str) + " " + train["MC_Answer"].astype(str)).tolist()
labels = train["binary_label"].tolist()
dataset = Dataset.from_dict({"text": texts, "label": labels})

# Tokenizer
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
dataset = dataset.map(lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=128), batched=True)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Train/val split
dataset = dataset.train_test_split(test_size=0.2, seed=42)

# Model
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Training arguments
training_args = TrainingArguments(
    output_dir="./binary_model",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_steps=100,
    save_strategy="no",
    disable_tqdm=False,
    report_to="none"  # avoids trying to log to WandB/other
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
)

# Train
trainer.train()

# After trainer.train()
model.save_pretrained("./binary_model_final")
tokenizer.save_pretrained("./binary_model_final")

# Check CUDA
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.current_device(), torch.cuda.get_device_name(0))


Map:   0%|          | 0/37083 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_138/507155439.py:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
1,0.691300
2,0.698000
3,0.687300
4,0.704600
5,0.728600
6,0.673200
7,0.676100
8,0.660300
9,0.672300
10,0.785800


CUDA available: True
Device: 0 Tesla T4


In [4]:
# After trainer.train()
model.save_pretrained("./binary_model_final")
tokenizer.save_pretrained("./binary_model_final")



('./binary_model_final/tokenizer_config.json',
 './binary_model_final/special_tokens_map.json',
 './binary_model_final/vocab.txt',
 './binary_model_final/added_tokens.json',
 './binary_model_final/tokenizer.json')

In [6]:

# Paths
MODEL_NAME = "distilbert-base-uncased"
MODEL_DIR = "./binary_model"  # same as output_dir used in training
TEST_CSV = "/kaggle/input/map-charting-student-math-misunderstandings/train_Clean.csv"

# Load test data
test_df = pd.read_csv(TEST_CSV)

# Prepare text
texts = (test_df["QuestionText"].astype(str) + " " + test_df["MC_Answer"].astype(str)).tolist()

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("./binary_model_final")
model = AutoModelForSequenceClassification.from_pretrained("./binary_model_final")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
binary_model = model

# Tokenize
encodings = tokenizer(texts, truncation=True, padding="max_length", max_length=128, return_tensors="pt")
input_ids = encodings["input_ids"].to(device)
attention_mask = encodings["attention_mask"].to(device)
import torch
from tqdm import tqdm

batch_size = 32  # reduce if still OOM
binary_preds = []

binary_model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
binary_model.to(device)

# Move encodings to CPU temporarily if needed
input_ids_all = encodings["input_ids"]
attention_mask_all = encodings["attention_mask"]

for i in tqdm(range(0, len(input_ids_all), batch_size)):
    batch_input_ids = input_ids_all[i:i+batch_size].to(device)
    batch_attention_mask = attention_mask_all[i:i+batch_size].to(device)

    with torch.no_grad():
        outputs = binary_model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
        batch_preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        binary_preds.extend(batch_preds)

# Add predictions to dataframe
train["binary_pred"] = binary_preds
print(train[["QuestionText", "MC_Answer", "binary_pred"]].head())


100%|██████████| 1159/1159 [02:21<00:00,  8.17it/s]


                                        QuestionText          MC_Answer  \
0  What fraction of the shape is not shaded? Give...  \( \frac{1}{3} \)   
1  What fraction of the shape is not shaded? Give...  \( \frac{1}{3} \)   
2  What fraction of the shape is not shaded? Give...  \( \frac{1}{3} \)   
3  What fraction of the shape is not shaded? Give...  \( \frac{1}{3} \)   
4  What fraction of the shape is not shaded? Give...  \( \frac{1}{3} \)   

   binary_pred  
0            1  
1            1  
2            1  
3            1  
4            1  


In [23]:
train[train['binary_pred']==0].head()

,row_id,QuestionId,QuestionText,MC_Answer,StudentExplanation,Category,Misconception,Category Misconception,binary_label,binary_pred,ternary_label,ternary_pred
2348,2348.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{3}{6} \),! simplefined it and that was what i got,False_Neither,NaN,False_Neither- NA,0,0,2,0
2349,2349.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{3}{6} \),2-Jan,False_Neither,NaN,False_Neither- NA,0,0,2,0
2350,2350.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{3}{6} \),3 IS SHADED AND 6 IS NOT,False_Misconception,WNB,False_Misconception- WNB,0,0,0,0
2351,2351.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{3}{6} \),3 OUT OF 6 SPOTS ARE NOT SHADED SO THE FRACTIO...,False_Misconception,WNB,False_Misconception- WNB,0,0,0,1
2352,2352.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{3}{6} \),3 are not shaded and 6 are shaded,False_Misconception,WNB,False_Misconception- WNB,0,0,0,0


In [8]:
def ternary_mapper(x):
    # Take the last part after splitting by "_"
    last_part = str(x).split("_")[-1].lower()
    
    if last_part == "correct":
        return 1
    elif last_part == "neither":
        return 2
    elif last_part == "misconception":
        return 0
    else:
        return -1

# Apply to the column
train["ternary_label"] = train["Category"].apply(ternary_mapper)


In [9]:
train.head()

,row_id,QuestionId,QuestionText,MC_Answer,StudentExplanation,Category,Misconception,Category Misconception,binary_label,binary_pred,ternary_label
0,0.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),0ne third is equal to tree nineth,True_Correct,NaN,True_Correct- NA,1,1,1
1,1.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 / 3 because 6 over 9 is 2 thirds and 1 third...,True_Correct,NaN,True_Correct- NA,1,1,1
2,2.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),"1 3rd is half of 3 6th, so it is simplee to un...",True_Neither,NaN,True_Neither- NA,1,1,2
3,3.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 goes into everything and 3 goes into nine,True_Neither,NaN,True_Neither- NA,1,1,2
4,4.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 out of every 3 isn't coloured,True_Correct,NaN,True_Correct- NA,1,1,1


In [26]:
train_valid = train[train["ternary_label"].isin([0,1,2])].copy()

def make_ternary_dataset(df):
    # Concatenate MC_Answer + StudentExplanation + binary prediction as string
    df["input_text"] = df["StudentExplanation"].astype(str) + " BinaryPred:" + df["binary_pred"].astype(str)
    texts = df["input_text"].tolist()
    labels = df["ternary_label"].tolist()
    return Dataset.from_dict({"text": texts, "label": labels})

ds_ternary = make_ternary_dataset(train_valid)


In [52]:
ds_ternary['train']['label']

Column([tensor(1), tensor(0), tensor(2), tensor(0), tensor(0)])

In [27]:
assert all(l in [0,1,2] for l in ds_ternary["label"]), "Invalid labels detected!"


we will need to train it on more epochs

In [28]:
# Tokenize
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

ds_ternary = ds_ternary.map(tokenize_function, batched=True)
ds_ternary.set_format("torch")
ds_ternary = ds_ternary.train_test_split(test_size=0.2, seed=42)

# ----------------------------
# 6️⃣ Train ternary model
# ----------------------------
def train_ternary_model(dataset, num_labels=3, output_dir="./ternary_model"):
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)

    training_args = TrainingArguments(
        output_dir=output_dir,
        do_eval=True,
        per_device_train_batch_size=16,  # reduce if OOM occurs
        per_device_eval_batch_size=16,
        num_train_epochs=1,
        weight_decay=0.01,
        logging_steps=100,
        save_strategy="no",
        disable_tqdm=False,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=ds_ternary['train'],
        eval_dataset=ds_ternary['test'],  # ideally use a real validation split
        tokenizer=tokenizer
    )

    trainer.train()

    # Save model + tokenizer
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    return trainer

ternary_trainer = train_ternary_model(ds_ternary, num_labels=3, output_dir="./ternary_model")

# ----------------------------
# 9️⃣ Confirm CUDA
# ----------------------------
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.current_device(), torch.cuda.get_device_name(0))


Map:   0%|          | 0/36696 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_138/1484366890.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
100,0.671400
200,0.566200
300,0.522700
400,0.492500
500,0.482600
600,0.440700
700,0.456700
800,0.447700
900,0.431100


CUDA available: True
Device: 0 Tesla T4


In [29]:
# After trainer.train()
model.save_pretrained("./ternary_model_final")
tokenizer.save_pretrained("./ternary_model_final")



('./ternary_model_final/tokenizer_config.json',
 './ternary_model_final/special_tokens_map.json',
 './ternary_model_final/vocab.txt',
 './ternary_model_final/added_tokens.json',
 './ternary_model_final/tokenizer.json')

In [30]:

# Paths
MODEL_NAME = "distilbert-base-uncased"
MODEL_DIR = "./ternary_model_final"  # same as output_dir used in training
TEST_CSV = "/kaggle/input/map-charting-student-math-misunderstandings/train_Clean.csv"

# Load test data
test_df = train

# Prepare text
texts = (test_df["StudentExplanation"].astype(str) + " BinaryPred:" + test_df["binary_pred"].astype(str)).tolist()

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
binary_model = model

# Tokenize
encodings = tokenizer(texts, truncation=True, padding="max_length", max_length=128, return_tensors="pt")
input_ids = encodings["input_ids"].to(device)
attention_mask = encodings["attention_mask"].to(device)
import torch
from tqdm import tqdm

batch_size = 32  # reduce if still OOM
binary_preds = []

binary_model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
binary_model.to(device)

# Move encodings to CPU temporarily if needed
input_ids_all = encodings["input_ids"]
attention_mask_all = encodings["attention_mask"]

for i in tqdm(range(0, len(input_ids_all), batch_size)):
    batch_input_ids = input_ids_all[i:i+batch_size].to(device)
    batch_attention_mask = attention_mask_all[i:i+batch_size].to(device)

    with torch.no_grad():
        outputs = binary_model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
        batch_preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        binary_preds.extend(batch_preds)

# Add predictions to dataframe
train["ternary_pred"] = binary_preds
print(train[["StudentExplanation", "binary_pred", "ternary_pred"]].head())


100%|██████████| 1159/1159 [02:21<00:00,  8.20it/s]

                                  StudentExplanation  binary_pred  \
0                  0ne third is equal to tree nineth            1   
1  1 / 3 because 6 over 9 is 2 thirds and 1 third...            1   
2  1 3rd is half of 3 6th, so it is simplee to un...            1   
3        1 goes into everything and 3 goes into nine            1   
4                    1 out of every 3 isn't coloured            1   

   ternary_pred  
0             1  
1             1  
2             1  
3             1  
4             1  


In [60]:
len(train[train["ternary_label"]==2])

11807

In [45]:
train.head(30)

,row_id,QuestionId,QuestionText,MC_Answer,StudentExplanation,Category,Misconception,Category Misconception,binary_label,binary_pred,ternary_label,ternary_pred
0,0.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),0ne third is equal to tree nineth,True_Correct,NaN,True_Correct- NA,1,1,1,1
1,1.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 / 3 because 6 over 9 is 2 thirds and 1 third...,True_Correct,NaN,True_Correct- NA,1,1,1,1
2,2.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),"1 3rd is half of 3 6th, so it is simplee to un...",True_Neither,NaN,True_Neither- NA,1,1,2,1
3,3.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 goes into everything and 3 goes into nine,True_Neither,NaN,True_Neither- NA,1,1,2,1
4,4.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 out of every 3 isn't coloured,True_Correct,NaN,True_Correct- NA,1,1,1,1
5,5.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 third because 6 over 9 is 2 thirds and 1 thi...,True_Correct,NaN,True_Correct- NA,1,1,1,1
6,6.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1/3 and 3/9 are not shaded but they are equal ...,True_Correct,NaN,True_Correct- NA,1,1,1,1
7,7.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1/3 because 3/9 simplified divided by 3 would ...,True_Correct,NaN,True_Correct- NA,1,1,1,1
8,8.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1/3 because I worked out that 3/9 were not sha...,True_Correct,NaN,True_Correct- NA,1,1,1,1
9,9.0,31772.0,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1/3 because it 3 shadedd in not all of them,True_Neither,NaN,True_Neither- NA,1,1,2,1


In [61]:
train.to_csv('updated_math_misunnderstanding.csv', index=False)